In [1]:
from pathlib import Path as pt
import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from scipy.special import wofz
from scipy.optimize import curve_fit
from uncertainties import ufloat as uf, unumpy as unp

In [2]:
def optimizePlot(ax, xlabel, ylabel, yscale="linear", fmtString="{x:.1f}", Nlocator=5):
    ax.set(yscale="linear")
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)

    ax.minorticks_on()
    ax.tick_params(axis='both', labelsize=16)
    ax.tick_params(which='major', width=2)
    ax.tick_params(which='minor', width=1)
    ax.tick_params(which='major', length=7)
    ax.tick_params(which='minor', length=4)

    ax.xaxis.set_major_formatter(plticker.StrMethodFormatter(fmtString))
    loc = plticker.MaxNLocator(Nlocator)
    ax.xaxis.set_major_locator(loc)
    return ax

np.seterr(all="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def voigt_profile(x, mu, amp, sigma, gamma):
    z = ((x-mu)+1j*gamma)/(sigma*np.sqrt(2))
    real = wofz(z).real
    # norm = (sigma*np.sqrt(2*np.pi))
    return amp*real

def voigt_fwhm(sigma, gamma):
    fG = 2*sigma*np.sqrt(2*np.log(2))
    fL = gamma*2
    return 0.5346*fL + unp.sqrt(0.2166*fL**2 + fG**2)

In [36]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

freq = 453_521.8573
signal = 25
print(f"{freq=:.2e}")

wn, inten = np.genfromtxt("./CD+_in_MHz.dat").T
ax.plot(wn, inten, ".k", label="Exp.")

# sigma = 0
# gamma = 0.1
# fG = sigma*(2*np.sqrt(2*np.log(2)))
# fL = gamma*2

fG = 0.318
fL = 0.160
sigma = fG/(2*np.sqrt(2*np.log(2)))
gamma = fL/2


pop, pcov = curve_fit(voigt_profile, wn, inten, p0=[freq, signal, sigma, gamma])
perr = np.diag(np.sqrt(pcov))

parameters = unp.uarray(pop, perr)
print(f"{pop=}\n{perr=}")

freq_fit, signal_fit, sigma_fit, gamma_fit = parameters

fG_fit = sigma_fit*(2*np.sqrt(2*np.log(2)))
fL_fit = 2*gamma_fit

print(f"{fG_fit=:.3f}\n{fL_fit=:.3f}\n{freq_fit=:.3f}\n{signal_fit=:.3f}")

fV = voigt_fwhm(sigma_fit, gamma_fit)
# fV = 0.5346*fL_fit + unp.sqrt(0.2166*fL_fit**2 + fG_fit**2)

print(f"{fG_fit=:.3f}\n{fL_fit=:.3f}\n{freq_fit=:.3f}\n{signal_fit=:.3f}")
label = f"$v: ${freq_fit:.4f} MHz\nSignal: {signal_fit:.1f} %\n$f_G: ${fG_fit*1e3:.1f} KHz\n$f_L: ${fL_fit*1e3:.1f} KHz\n$f_V: ${fV*1e3:.1f} KHz"

ax.plot(wn, voigt_profile(wn, *pop), "-k", label=label)
ax.vlines(freq_fit.nominal_value, ymin=0, ymax=signal_fit.nominal_value, ls="--")
ax.hlines(signal_fit.nominal_value/2, xmin=freq_fit.nominal_value-fV.nominal_value/2, xmax=freq_fit.nominal_value+fV.nominal_value/2, ls="--")
ax = optimizePlot(ax, "Frequency(MHz)", "Signal (%)")
legend = ax.legend(fontsize=14)
plt.show()
savelocation = pt("./")
savename = "CD+_J01"
plt.savefig(f"{savelocation/savename}.png")
plt.savefig(f"{savelocation/savename}.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

freq=4.54e+05
pop=array([4.53521857e+05, 2.64031726e+01, 1.68075720e-01, 1.11476787e-02])
perr=array([0.00353148, 2.29466695, 0.01047121, 0.01704714])
fG_fit=0.396+/-0.025
fL_fit=0.022+/-0.034
freq_fit=453521.857+/-0.004
signal_fit=26.403+/-2.295
fG_fit=0.396+/-0.025
fL_fit=0.022+/-0.034
freq_fit=453521.857+/-0.004
signal_fit=26.403+/-2.295


In [33]:
fG = 0.252717675256927
fL = 0.264242580083222
sigma = fG/(2*np.sqrt(2*np.log(2)))
gamma = fL/2
fV = 0.5346*fL + np.sqrt(0.2166*fL**2 + fG**2)
print(f"{fV*1e3=:.3f}")

fV*1e3=422.316
